In [2]:
from sympy.abc import A, B, C

In [3]:
from sympy.logic.boolalg import to_cnf

In [4]:
to_cnf((~A & B & C) | (A & ~B & C) | (A & B & ~C))

(A | B) & (A | C) & (B | C) & (A | ~A) & (B | ~B) & (C | ~C) & (A | B | C) & (A | B | ~A) & (A | B | ~B) & (A | B | ~C) & (A | C | ~A) & (A | C | ~B) & (A | C | ~C) & (B | C | ~A) & (B | C | ~B) & (B | C | ~C) & (A | ~A | ~B) & (A | ~A | ~C) & (B | ~A | ~B) & (B | ~B | ~C) & (C | ~A | ~C) & (C | ~B | ~C) & (~A | ~B | ~C)

This is not great - it even contains reduntant clauses `A | ~A`.

In [5]:
from sympy.logic.boolalg import simplify_logic

In [6]:
simplify_logic((~A & B & C) | (A & ~B & C) | (A & B & ~C), form='cnf')

(A | B) & (A | C) & (B | C) & (~A | ~B | ~C)

Let's try 1 correct out of 3:

In [10]:
simplify_logic((A & ~B & ~C) | (~A & B & ~C) | (~A & ~B & C), form='cnf')

(A | B | C) & (~A | ~B) & (~A | ~C) & (~B | ~C)

In [9]:
from sympy.abc import D, E

1 of 5

In [11]:
simplify_logic((A & ~B & ~C & ~D & ~E) 
               | (~A & B & ~C & ~D & ~E) 
               | (~A & ~B & C & ~D & ~E)
               | (~A & ~B & ~C & D & ~E)
               | (~A & ~B & ~C & ~D & E), form='cnf')

(~A | ~B) & (~A | ~C) & (~A | ~D) & (~A | ~E) & (~B | ~C) & (~B | ~D) & (~B | ~E) & (~C | ~D) & (~C | ~E) & (~D | ~E) & (A | B | C | D | E)

2 of 5

In [13]:
from sympy.logic.boolalg import true, false
xs = [A, B, C, D, E]

clause = false

for i in range(len(xs)):
    for j in range(i+1, len(xs)):
        term = true
        for k, x in enumerate(xs):
            if k in (i, j):
                term &= x
            else:
                term &= ~x
        clause |= term

print(clause)
simplify_logic(clause, form='cnf')
        

(A & B & ~C & ~D & ~E) | (A & C & ~B & ~D & ~E) | (A & D & ~B & ~C & ~E) | (A & E & ~B & ~C & ~D) | (B & C & ~A & ~D & ~E) | (B & D & ~A & ~C & ~E) | (B & E & ~A & ~C & ~D) | (C & D & ~A & ~B & ~E) | (C & E & ~A & ~B & ~D) | (D & E & ~A & ~B & ~C)


(A | B | C | D) & (A | B | C | E) & (A | B | D | E) & (A | C | D | E) & (B | C | D | E) & (~A | ~B | ~C) & (~A | ~B | ~D) & (~A | ~B | ~E) & (~A | ~C | ~D) & (~A | ~C | ~E) & (~A | ~D | ~E) & (~B | ~C | ~D) & (~B | ~C | ~E) & (~B | ~D | ~E) & (~C | ~D | ~E)

# Conclusions

Read Raymond, and maybe use his great tools: https://rhettinger.github.io/einstein.html

What is enough to be observed from the above examples - while you may know (e.g. from solving sudoku with pycosat), that:

In [17]:
from itertools import combinations

def one_and_only_one(values):
    yield values
    for a, b in combinations(values, 2):
        yield [-a, -b]

Raymond has this with `Q(elements) == 1`. This is "all the elements in sum", times "pairs of negated elements". Easy - one must be true, but in any pair at least one must be false. But this generalizes to "k of N is true".

So, let's start with the negated part - if $k$ is true, that means in groups of $k+1$ at least one must be false. That's why we take all combinations $C(N, k+1)$ as terms of negated literals.

Now, what must be true - is the elements when we remove $(k-1)$ members. Of the remeining $N - k + 1$ elements, one is true. These are $C(N, N-k+1)$ terms.

In [18]:
def exactly_k(values, k):
    n = len(values)
    for t in combinations(values, n-k+1):
        yield list(t)
    for t in combinations(values, k+1):
        yield [-x for x in t]


The above function is general enought for `k==1`.

In [20]:
list(exactly_k([1, 2, 3], 1))

[[1, 2, 3], [-1, -2], [-1, -3], [-2, -3]]

And interestingly, even for `k==0`:

In [22]:
list(exactly_k([1, 2, 3], 0))

[[-1], [-2], [-3]]